train data -> N:51542, Y:7657 개

0, np.nan 값 다 드랍

매년 전기 에너지 사용량 달별로 더해서 컬럼 12개로 압축

log 변환

Y,N 그룹 별 각 컬럼간 상관도 ->pairplot

Y,N 그룹 별 각 컬럼의 평균값 barplot

Y,N 그룹 별 각 컬럼의 분포도 box plot

Y,N 그룹 별 각 컬럼의 분포도 dist plot

Y,N 그룹 별 건물용도 별 각 컬럼의 분포도 boxplot

각 컬럼의 NuLL값 주거용도 별 펑균값으로 채우고 예측 - >  정확도: 0.8371, 정밀도: 0.8426, 재현율: 0.1431,    F1: 0.2446, AUC:0.5685
너무 쉽게 N으로 판단(TN 엄청 많음) -> N의 0값의 비중이 너무 큼 ->  Y의 0을 그냥 N으로 판단하는듯 -> 0부터 어떻게 해야함!

In [1]:
#filename = 'finalized_model.sav'
#pickle.dump(model, open(filename, 'wb'))
 
# some time later...

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [1]:
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   

In [2]:
import pandas as pd
pd.set_option('display.max_row',500)
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.mixture import GaussianMixture
from sklearn.metrics import SCORERS
from xgboost import plot_importance
import pickle

In [3]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('RMSE: {0}'.format((rmse)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [4]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, r2_score, mean_absolute_error

def get_clf_eval(y_test , pred, typ):
    if typ=='classification':
        confusion = confusion_matrix( y_test, pred)
        accuracy = accuracy_score(y_test , pred)
        precision = precision_score(y_test , pred)
        recall = recall_score(y_test , pred)
        f1 = f1_score(y_test,pred)
        roc_auc = roc_auc_score(y_test, pred)
        print('오차 행렬')
        print(confusion)
        print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
        F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    if typ=='regression':
        r2 = r2_score(y_test, pred)
        mase = mean_absolute_error(y_test, pred)
        print('r2 score : {0}'.format(r2))
        print('mae score : {0}'.format(mase))
   
    

In [5]:
def visualize_silhouette(cluster_lists, X_features,cluster_labels): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        #clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        #cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n'+'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
        

In [6]:
def kmeans_plot(clusterDF,n_clusters,max_iter):

    kmeans = KMeans(n_clusters, init='k-means++', max_iter=200, random_state=0)
    cluster_labels = kmeans.fit_predict(clusterDF)
    col=clusterDF.columns
    clusterDF['kmeans_label']  = cluster_labels

#cluster_centers_ 는 개별 클러스터의 중심 위치 좌표 시각화를 위해 추출
    centers = kmeans.cluster_centers_
    unique_labels = np.unique(cluster_labels)
    markers=['o', 's', '^', 'P','D','H','x']

# 군집된 label 유형별로 iteration 하면서 marker 별로 scatter plot 수행. 
    for label in unique_labels:
        label_cluster = clusterDF[clusterDF['kmeans_label']==label]
        center_x_y = centers[label]
        plt.scatter(x=label_cluster[col[0]], y=label_cluster[col[1]], edgecolor='k', 
                    marker=markers[label] )
    
    # 군집별 중심 위치 좌표 시각화 
        plt.scatter(x=center_x_y[0], y=center_x_y[1], s=200, color='white',
                    alpha=0.9, edgecolor='k', marker=markers[label])
        plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k', edgecolor='k', 
                    marker='$%d$' % label)

    plt.show()

In [7]:
def plot_mean(df_n):
    index = np.array(df_n.mean().index)
    height = np.array(df_n.mean())
    plt.figure(figsize=(16,10))
    plt.bar(index, height)
    
def plot_dist(df):
    list_series = df.columns
    n=len(list_series)
    plt.figure(figsize=(25,6))
    fig , axes = plt.subplots(n//2+1,2,figsize=(15,40))
    axx = axes.flatten()
    for index,i in enumerate(list_series[:-2]):
        x=df[i]
        sns.distplot(x, kde=True, rug=False,ax=axx[index])
    
def elec_month(df_y):
    df_y_local = df_y.drop(['label','bldng_us'],axis=1)
    for i in range(1,13):
        col = '{0} month'.format(i)
        df_y_local[col]=df_y_local.iloc[:,(i-1)::12].sum(axis=1)
    df_y_local = df_y_local.iloc[:,-1:-13:-1]
    df_y_local = df_y_local.iloc[:,-1:-13:-1]
    df_y_local['bldng_us']=df_y['bldng_us']
    df_y_local['label']=df_y['label']
    return df_y_local

def plot_box1(df,label,h=None):
    list_series = df.columns
    n=len(list_series)
    #fig , axes = plt.subplots(4,n//4+1,figsize=(15,15))
    #axx = axes.flatten()
    if(h==None):
        fig , axes = plt.subplots(4,n//4+1,figsize=(15,15))
        axx = axes.flatten()
        for index,i in enumerate(list_series[:-2]):
            sns.boxplot(i, label, data=df,ax=axx[index])
            #sns.stripplot(x, 'bldng_us',hue=label, data=df, jitter=True, color="0.4",ax=axx[index])
    else:
        fig , axes = plt.subplots(4,n//4+1,figsize=(15,150))
        axx = axes.flatten()
        for index,i in enumerate(list_series[:-2]):
            sns.boxplot(i, label,hue=h ,data=df,ax=axx[index])
            #sns.stripplot(x, 'bldng_us',hue=label, data=df, jitter=True, color="0.4",ax=axx[index])
        

def plot_box2(df,label):
    list_series = df.columns
    n=len(list_series)
    fig , axes = plt.subplots(4,n//4+1,figsize=(15,150))
    axx = axes.flatten()
    for index,i in enumerate(list_series[:-2]):
        sns.boxplot(i, label,hue='bldng_us', data=df,ax=axx[index])
        #sns.stripplot(x, 'bldng_us',hue=label, data=df, jitter=True, color="0.4",ax=axx[index])
        
def get_outlier(df=None, column=None, weight=1.5):
    # fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함. 
    #fraud = df[df['label']==1][column]
    fraud = df[column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    # IQR을 구하고, IQR에 1.5를 곱하여 최대값과 최소값 지점 구함. 
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    # 최대값 보다 크거나, 최소값 보다 작은 값을 아웃라이어로 설정하고 DataFrame index 반환. 
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index

def fill_elec(df):
    list_ = list(df.columns[:-2])
    for i in list_:
        df[i] = df[i].apply(lambda x: np.nan if x==0 else x)
        #df[i].fillna(df[i].mean(), inplace=True)
        df=df.dropna()
    return df

def circum_plot(df,year,drop=False):
    df_elec = df.iloc[:,18:137:2]
    if drop==True:
        list_ = list(df_elec.columns)
        for i in list_:
            df_elec[i] = df_elec[i].apply(lambda x: np.nan if x==0 else x)
    df_elec=df_elec.dropna()
    df_elec = df_elec.apply(lambda x: np.log10(1+x))
    #list_col = list(df.columns)
    #index = list_col.index('bldng_cnt')
    #df_cir=df.iloc[:,index:index+3]
    df_cir=df.iloc[:,12:17]
    list_cir = list(df_cir.columns)
    df_elec = pd.concat([df_elec,df_cir],axis=1)
    df_elec['fr_yn']=df['fr_yn']
    #df_elec['mon'] = df['dt_of_fr'].apply(lambda x: int(x.split('-')[1]))
    col='ele_engry_us_'
    year=str(year)
    for i in range(1,11):
        if(i<=9):
            mon='0'+str(i)
        else:
            mon=str(i)
        col_a =col+year+mon
        fig =plt.figure(figsize=(10,12))
        ax1 = fig.add_subplot(3, 2, 1)
        ax2 = fig.add_subplot(3, 2, 2)
        ax3 = fig.add_subplot(3, 2, 3)
        ax4 = fig.add_subplot(3, 2, 4)
        ax5 = fig.add_subplot(3, 2, 5)
        list_ax=[ax1, ax2, ax3, ax4, ax5]
        for index,j in enumerate(list_cir):
            df_elec_1=df_elec[df_elec['mon']==i][[col_a,j]]
            list_ax[index].set_xlabel(col_a)
            list_ax[index].set_ylabel(j)
            list_ax[index].set_title(year)
            list_ax[index].scatter(df_elec_1.iloc[:,0],df_elec_1.iloc[:,1])
            
def building_plot(df,year,mon,drop=False):
    df_elec = df.iloc[:,18:137:2]
    list_col = list(df.columns)
    index = list_col.index('bldng_cnt')
    df_cir=df.iloc[:,index:index+3]
    df_elec = pd.concat([df_elec,df_cir],axis=1)
    list_cir = list(df_cir.columns)
    if drop==True:
        list_ = list(df_elec.columns[:-4])
        for i in list_:
            df_elec[i] = df_elec[i].apply(lambda x: np.nan if x==0 else x)
            
    df_elec=df_elec.dropna()
    
    df_elec.iloc[:,:-4] = df_elec.iloc[:,:-4].apply(lambda x: np.log10(1+x))
    
    col='ele_engry_us_'
    year=str(year)
    col_a =col+year+mon
    for index,j in enumerate(list_cir):
        df_elec_1=df_elec[[col_a,j]]
        visualize_silhouette([2,3],df_elec_1)
        
def df_prepro(df,drop):
    df=fill_circum(df)
    list_col=list(df.columns)
    df_elec = df.iloc[:,18:137:2]
    list_x = list(df_elec.columns)
    index2 = list_x.index('ele_engry_us_201401')
    df_elec=df_elec.iloc[:,index2:]
    index = list_col.index('bldng_cnt')
    df_cir=df.iloc[:,index:index+3]
    df_cir2 = df.iloc[:,12:17]
    df_cir2.drop('prcpttn',axis=1,inplace=True)
    df_elec = pd.concat([df_cir, df_cir2,df_elec],axis=1)
    df_elec = df_elec.replace([np.inf, -np.inf], np.nan)
    
    a=get_outlier(df_cir,'bldng_cnt').tolist()
    b=get_outlier(df_cir,'bldng_ar').tolist()
    c=get_outlier(df_cir,'ttl_ar').tolist()
    index_list=a+b+c
    index_list = np.unique(index_list)
    index_list=np.array(index_list)
    index_list=index_list.reshape(-1,)
    index_list=np.unique(index_list)
    index_list.sort()
    
    q=pd.Series(index_list)
    w=pd.DataFrame(q,columns=['col1'])
    w.index=index_list
    x=w.index
    df_elec.drop(x,axis=0,inplace=True)
    df_elec = df_elec.reset_index(drop=True)
    
    df_list=[]
    
    if drop==True:
        list_x = list(df_elec.columns)
        index2 = list_x.index('ele_engry_us_201401')
        list_ = list(df_elec.columns[index2:])
        for i in list_:
            df_elec[i] = df_elec[i].apply(lambda x: np.nan if x==0 else x)
            df_elec[i] = df_elec[i].replace([np.nan],'nan')
    
    for i in df_elec.iloc[:,7:].columns:
        x=df_elec[(df_elec[i]!='nan')]
        x=x[x[i]>=0]
        y=x.iloc[:,0:7]
        x[i]=x[i].apply(lambda x: np.log10(1+x))
        y[i]=x[i]
        df_list.append(y)
            
    #df_elec=df_elec.dropna()
    
    #df_elec.iloc[:,7:] = df_elec.iloc[:,7:].apply(lambda x: np.log10(1+x))
    return df_list

def df_prepro_gas(df,drop):
    df=fill_circum(df)
    list_col=list(df.columns)
    df_elec = df.iloc[:,17:137:2]
    list_x = list(df_elec.columns)
    index2 = list_x.index('gas_engry_us_201401')
    df_elec=df_elec.iloc[:,index2:]
    index = list_col.index('bldng_cnt')
    df_cir=df.iloc[:,index:index+3]
    df_cir2 = df.iloc[:,12:17]
    df_cir2.drop('prcpttn',axis=1,inplace=True)
    df_elec = pd.concat([df_cir, df_cir2,df_elec],axis=1)
    df_elec = df_elec.replace([np.inf, -np.inf], np.nan)
    
    a=get_outlier(df_cir,'bldng_cnt').tolist()
    b=get_outlier(df_cir,'bldng_ar').tolist()
    c=get_outlier(df_cir,'ttl_ar').tolist()
    index_list=a+b+c
    index_list = np.unique(index_list)
    index_list=np.array(index_list)
    index_list=index_list.reshape(-1,)
    index_list=np.unique(index_list)
    index_list.sort()
    
    q=pd.Series(index_list)
    w=pd.DataFrame(q,columns=['col1'])
    w.index=index_list
    x=w.index
    df_elec.drop(x,axis=0,inplace=True)
    df_elec = df_elec.reset_index(drop=True)
    
    if drop==True:
        list_x = list(df_elec.columns)
        index2 = list_x.index('gas_engry_us_201401')
        list_ = list(df_elec.columns[index2:])
        for i in list_:
            df_elec[i] = df_elec[i].apply(lambda x: np.nan if x==0 else x)
            df_elec[i] = df_elec[i].replace([np.nan],'nan')
            
    df_list=[]
            
    for i in df_elec.iloc[:,7:].columns:
        x=df_elec[(df_elec[i]!='nan')]
        x=x[x[i]>=0]
        y=x.iloc[:,0:7]
        x[i]=x[i].apply(lambda x: np.log10(1+x))
        y[i]=x[i]
        df_list.append(y)
            
    #df_elec=df_elec.dropna()
    
    #df_elec.iloc[:,7:] = df_elec.iloc[:,7:].apply(lambda x: np.log10(1+x))
    return df_list

def sil_plot(X_features, cluster_labels):
        n_cluster = len(np.unique(cluster_labels))
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        print(sil_avg)
        
def gmm_plot(clusterDF):
    col=clusterDF.columns
    gmm = GaussianMixture(n_components=3, random_state=0).fit(clusterDF)
    cluster_labels = gmm.predict(clusterDF)

    clusterDF['meanshift_label']  = cluster_labels
    #centers = gmm.cluster_centers_
    unique_labels = np.unique(cluster_labels)
    markers=['o', 's', '^', 'x', '*']

    for label in unique_labels:
        label_cluster = clusterDF[clusterDF['meanshift_label']==label]
        #center_x_y = centers[label]
    # 군집별로 다른 marker로 scatter plot 적용
        plt.scatter(x=label_cluster[col[0]], y=label_cluster[col[1]], edgecolor='k', 
                    marker=markers[label] )
    
    plt.show()
    
def make_df(df_elec):
    list_year=['2017','2018']
    list_mon=['01','02','03','04','05','06','07','08','09','10','11','12']
    list_df =[]
    X_list=[]
    Y_list=[]
    for i in list_year:
        for j in list_mon:
            col='ele_engry_us_' + i + j
            a=df_elec.loc[:,[col,'bldng_cnt','bldng_ar','ttl_ar','tmprtr','wnd_spd','wnd_drctn','hmdt']]
            list_df.append(a)
            X_list.append(a.iloc[:,1:])
            Y_list.append(a.iloc[:,0])
    return list_df, X_list, Y_list

def make_df_gas(df_elec):
    list_year=['2014','2015','2016','2017','2018']
    list_mon=['01','02','03','04','05','06','07','08','09','10','11','12']
    list_df =[]
    X_list=[]
    Y_list=[]
    for i in list_year:
        for j in list_mon:
            col='gas_engry_us_' + i + j
            a=df_elec.loc[:,[col,'bldng_cnt','bldng_ar','ttl_ar','tmprtr','wnd_spd','wnd_drctn','hmdt']]
            list_df.append(a)
            X_list.append(a.iloc[:,1:])
            Y_list.append(a.iloc[:,0])
    return list_df, X_list, Y_list


def fill_circum(df):
    #df['dt_of_fr'] = df['dt_of_fr'].apply(lambda x: int(x.split('-')[1]))
    list=['tmprtr','wnd_spd','wnd_drctn','hmdt']
    for i in list:
        df[i]=df[i].replace([np.inf,-np.inf],np.nan)
        df[i]=df[i].fillna(df.groupby(by='fr_yn')[i].transform('mean'))

    return df

def df_prepro_vali(df,drop):
    df=df.replace([np.inf, -np.inf], np.nan)
    df=fill_circum(df)
    list_col=list(df.columns)
    df_elec = df.iloc[:,18:137:2]
    list_x = list(df_elec.columns)
    index2 = list_x.index('ele_engry_us_201701')
    df_elec=df_elec.iloc[:,index2:]
    index = list_col.index('bldng_cnt')
    df_cir=df.iloc[:,index:index+3]
    df_cir2 = df.iloc[:,12:17]
    df_cir2.drop('prcpttn',axis=1,inplace=True)
    df_elec = pd.concat([df_cir, df_cir2,df_elec],axis=1)
    #df_elec = df_elec.replace([np.inf, -np.inf], np.nan)
    
    df_elec = df_elec.reset_index(drop=True)
    
    if drop==True:
        list_x = list(df_elec.columns)
        index2 = list_x.index('ele_engry_us_201701')
        list_ = list(df_elec.columns[index2:])
        for i in list_:
            df_elec[i] = df_elec[i].apply(lambda x: np.nan if x==0 else x)
    
    df_elec.iloc[:,9:] = df_elec.iloc[:,9:].apply(lambda x: np.log10(1+x))
    return df_elec

def plot_heat(df,x,y):
    z=x[:]
    z.extend(y)
    df2=df[z]
    df2.dropna(inplace = True)
    df_size = df2.pivot_table(index=x, columns=y, aggfunc="size")
    df_size['sum']=df_size.apply((lambda x:(x.sum())),axis=1)
    df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False).iloc[1]/x['sum']),axis=1)
    df_size['score']=df_size.apply( (lambda x :(x['sum']*np.log10(x['percentage'])) if x['sum']!=1 else np.nan ),axis=1 )
    return (x,y), df_size['score'].mean(),df_size
    #fig = plt.figure(figsize=(20,20))
    #plt.title('{0}'.format(i))
    #sns.heatmap(df_size, cmap=sns.light_palette("gray", as_cmap=True), annot=True, fmt=".0f")
    #return fig
    
def find_2(df_nomi,col):
    cols = df_nomi.columns[1:]
    n = len(cols)
    scores=[]
    results=[]
    for j in cols:
        if j!=col:
            for i in cols:
                if ((i!=j) & (i!=col)):
                    result = plot_heat(df,[j,i],[col])
                    results.append(result)
                    scores.append(result[1])
    arr_scores = np.array(scores)
    index = arr_scores.argsort()[::-1][0]
    return results[index][1],results[index][2]

In [8]:
df = pd.read_csv('PJT002_train_03.csv')
#df['mon'] = df['dt_of_fr'].apply(lambda x: int(x.split('-')[1]))
df.columns[8:]
df.isna().sum().sort_values(ascending=False)

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


lw_13121011                     58926
lw_13121010                     58926
lw_13111110                     58918
lw_13101410                     58916
lw_13101110                     58908
lw_13141011                     58908
lw_13101310                     58907
lw_13141010                     58907
lw_13131110                     58903
lw_13131010                     58903
lw_13101210                     58902
lw_13101211                     58901
lw_13111010                     58899
lw_13101010                     58897
prcpttn                         53629
trgt_crtr                       51022
fr_fghtng_fclt_spcl_css_6_yn    50762
fr_fghtng_fclt_spcl_css_5_yn    50762
dngrs_thng_yn                   49447
slf_fr_brgd_yn                  49447
blk_dngrs_thng_mnfctr_yn        49447
us_yn                           49447
cltrl_hrtg_yn                   49447
bldng_us_clssfctn               29343
bldng_us                        27677
bldng_archtctr                  27665
ele_engry_us

In [10]:
df_nomi = df[['fr_yn','bldng_cnt','bldng_us','bldng_archtctr','bldng_us_clssfctn','jmk','rgnl_ar_nm','rgnl_ar_nm2','lnd_us_sttn_nm','rd_sd_nm','emd_nm','mlt_us_yn','ttl_grnd_flr','ttl_dwn_flr','floors','wnd_dr8']]
df_nomi.to_csv('df_nomi')
df_nomi.head()
df_nomi.isna().sum().sort_values(ascending=False)
#df['bldng_cnt'].value_counts().index
#'bldng_yrs'
#'bldng_cnt

bldng_us_clssfctn    29343
bldng_us             27677
bldng_archtctr       27665
floors               11006
ttl_dwn_flr          11005
ttl_grnd_flr         10210
lnd_us_sttn_nm        1776
rd_sd_nm              1491
rgnl_ar_nm2           1491
rgnl_ar_nm            1491
emd_nm                   4
wnd_dr8                  0
mlt_us_yn                0
jmk                      0
bldng_cnt                0
fr_yn                    0
dtype: int64

In [161]:
x=df_nomi[(df_nomi['bldng_us'].isna())&(df_nomi['bldng_us_clssfctn'].isna())]
x.shape
          

(27672, 16)

In [162]:
x=df_nomi[(df_nomi['bldng_cnt'].isna())&(df_nomi['bldng_archtctr'].isna())]
x.shape

(0, 16)

In [11]:
cols = df_nomi.columns[1:]
n = len(cols)
names=[]
means=[]
frames=[]
for i in range(n):
    x=cols[i]
    for j in range(n):
        if i!=j:
            y=cols[j]
            name, mean, frame = plot_heat(df_nomi,[x],[y])
            names.append(name)
            means.append(mean)
            frames.append(frame)
arr_names = np.array(names)
arr_means = np.array(means)
scores = np.sort(arr_means)[::-1]
rank = arr_names[np.argsort(arr_means)[::-1]]
rank_score=[]
for index, i in enumerate(rank):
    rank_score.append(np.append(i,scores[index]))
rank_score

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[array(['emd_nm', 'mlt_us_yn', '-0.24692934019961635'], dtype='<U32'),
 array(['floors', 'mlt_us_yn', '-0.39224990952778743'], dtype='<U32'),
 array(['ttl_grnd_flr', 'mlt_us_yn', '-0.3956385615562995'], dtype='<U32'),
 array(['bldng_cnt', 'mlt_us_yn', '-0.8576221231834033'], dtype='<U32'),
 array(['emd_nm', 'rgnl_ar_nm2', '-1.2936093841622944'], dtype='<U32'),
 array(['floors', 'rgnl_ar_nm2', '-1.4807781625640881'], dtype='<U32'),
 array(['ttl_grnd_flr', 'rgnl_ar_nm2', '-1.4974952100268535'], dtype='<U32'),
 array(['lnd_us_sttn_nm', 'mlt_us_yn', '-1.88260512429932'], dtype='<U32'),
 array(['bldng_us', 'mlt_us_yn', '-1.971535385085932'], dtype='<U32'),
 array(['ttl_dwn_flr', 'mlt_us_yn', '-2.7162778714708025'], dtype='<U32'),
 array(['jmk', 'mlt_us_yn', '-3.163236033728777'], dtype='<U32'),
 array(['rgnl_ar_nm', 'mlt_us_yn', '-3.59501893856273'], dtype='<U32'),
 array(['bldng_archtctr', 'mlt_us_yn', '-3.699336538140042'], dtype='<U32'),
 array(['rgnl_ar_nm2', 'mlt_us_yn', '-4.3051293692

In [12]:
candidate=[]
n=len(rank_score)
for i in range(n):
    x1=rank_score[i][0]
    y1=rank_score[i][1]
    z1=rank_score[i][2]
    for j in range(n):
        x2=rank_score[j][0]
        y2=rank_score[j][1]
        z2=rank_score[j][2]
        if y1==x2:
            for k in range(n):
                x3=rank_score[k][0]
                y3=rank_score[k][1]
                z3=rank_score[k][2]
                if y2==x3:
                    if y3==x1:
                        candidate.append((x1,x2,x3,z1,z2,z3,1.0/(1.0/float(z1)+1.0/float(z2)+1.0/float(z3))))                             

score_list = np.array([i[3] for i in candidate])
index = np.argsort(score_list)[::-1]
arr_candi = np.array(candidate)
arr_candi[index]

array([['mlt_us_yn', 'bldng_archtctr', 'wnd_dr8', ...,
        '-1041.531700216882', '-10.277184238721162',
        '-10.166291922705293'],
       ['mlt_us_yn', 'bldng_archtctr', 'emd_nm', ...,
        '-2806.2642072983163', '-0.24692934019961635',
        '-0.246901442447452'],
       ['mlt_us_yn', 'bldng_archtctr', 'rd_sd_nm', ...,
        '-771.1589473156521', '-6.3663675433017595',
        '-6.310205962171983'],
       ...,
       ['emd_nm', 'mlt_us_yn', 'bldng_cnt', ..., '-14464.214633473492',
        '-1075.1561701439425', '-0.24686842786179652'],
       ['emd_nm', 'mlt_us_yn', 'wnd_dr8', ..., '-16971.19455047297',
        '-12851.550653260429', '-0.24692100318808072'],
       ['emd_nm', 'mlt_us_yn', 'rgnl_ar_nm2', ..., '-220.155941561101',
        '-5441.998381622332', '-0.2466415130512681']], dtype='<U20')

In [13]:
#지정되지 않음이 그냥 너무 많음
df2=df_nomi[['wnd_dr8','rgnl_ar_nm2']]
df2.dropna(inplace = True)
df_size = df2.pivot_table(index="wnd_dr8", columns="rgnl_ar_nm2", aggfunc="size")
df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False).iloc[0]/x.sum()),axis=1)
df_size

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


rgnl_ar_nm2,개발제한구역,계획관리지역,관리지역,근린상업지역,농림지역,보전관리지역,보전녹지지역,생산관리지역,생산녹지지역,일반공업지역,일반상업지역,자연녹지지역,자연환경보전지역,제1종일반주거지역,제2종일반주거지역,제3종일반주거지역,준공업지역,준주거지역,지정되지않음,percentage
wnd_dr8,,,,,,,,,,,,,,,,,,,,
E,1.0,12.0,NaN,1.0,16.0,8.0,5.0,1.0,2.0,NaN,4.0,31.0,NaN,4.0,11.0,2.0,4.0,2.0,6613.0,0.984517
N,3.0,24.0,2.0,2.0,24.0,20.0,20.0,8.0,5.0,2.0,13.0,53.0,8.0,21.0,54.0,6.0,3.0,7.0,15169.0,0.982194
NE,NaN,7.0,NaN,NaN,4.0,3.0,3.0,3.0,2.0,2.0,6.0,18.0,1.0,4.0,10.0,1.0,NaN,1.0,3266.0,0.980486
NW,NaN,17.0,NaN,NaN,10.0,16.0,3.0,4.0,1.0,NaN,10.0,33.0,12.0,16.0,25.0,5.0,3.0,8.0,9331.0,0.982831
S,3.0,15.0,2.0,NaN,7.0,10.0,2.0,6.0,1.0,4.0,13.0,24.0,3.0,8.0,15.0,2.0,NaN,3.0,7134.0,0.983729
SE,NaN,4.0,NaN,NaN,3.0,2.0,1.0,2.0,1.0,NaN,1.0,7.0,6.0,2.0,10.0,NaN,NaN,3.0,2180.0,0.981098
SW,NaN,5.0,NaN,NaN,6.0,6.0,3.0,2.0,1.0,2.0,5.0,25.0,1.0,4.0,7.0,1.0,1.0,2.0,3068.0,0.977381
W,1.0,11.0,NaN,2.0,10.0,18.0,NaN,10.0,4.0,1.0,10.0,42.0,8.0,22.0,18.0,2.0,3.0,5.0,9942.0,0.983480


In [14]:
#지정되지 않음이 그냥 너무 많음
df2=df_nomi[['ttl_dwn_flr','rgnl_ar_nm2']]
df2.dropna(inplace = True)
df_size = df2.pivot_table(index="ttl_dwn_flr", columns="rgnl_ar_nm2", aggfunc="size")
df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False).iloc[0]/x.sum()),axis=1)
df_size

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


rgnl_ar_nm2,개발제한구역,계획관리지역,관리지역,근린상업지역,농림지역,보전관리지역,보전녹지지역,생산관리지역,생산녹지지역,일반공업지역,일반상업지역,자연녹지지역,자연환경보전지역,제1종일반주거지역,제2종일반주거지역,제3종일반주거지역,준공업지역,준주거지역,지정되지않음,percentage
ttl_dwn_flr,,,,,,,,,,,,,,,,,,,,
0.0,6.0,39.0,2.0,2.0,45.0,36.0,21.0,16.0,9.0,7.0,46.0,145.0,21.0,55.0,83.0,6.0,4.0,23.0,43544.0,0.987168
1.0,1.0,2.0,NaN,2.0,3.0,1.0,1.0,NaN,1.0,1.0,3.0,16.0,NaN,5.0,19.0,5.0,NaN,3.0,2711.0,0.977289
2.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,NaN,NaN,NaN,227.0,0.953782
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,75.0,0.974026
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,39.0,0.886364
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,0.969697
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,20.0,0.909091
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,1.000000
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,1.000000


In [15]:
#쓸만한듯
#둘다 널인게 너무 많음
df2=df_nomi[['bldng_us','bldng_us_clssfctn']]
df2.dropna(inplace = True)
df_size = df2.pivot_table(index="bldng_us", columns="bldng_us_clssfctn", aggfunc="size")
df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False).iloc[0]/x.sum()),axis=1)
df_size

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


bldng_us_clssfctn,공공용,공업용,기타,농수산용,문교사회용,상업용,주거용,percentage
bldng_us,,,,,,,,
공동주택,NaN,NaN,3.0,NaN,NaN,71.0,858.0,0.920601
공장,NaN,1032.0,29.0,NaN,NaN,5.0,1.0,0.967198
관광휴게시설,NaN,NaN,NaN,NaN,20.0,8.0,NaN,0.714286
교육연구및복지시설,NaN,NaN,NaN,NaN,16.0,NaN,NaN,1.000000
교육연구시설,NaN,NaN,1.0,NaN,174.0,3.0,NaN,0.977528
근린생활시설,NaN,NaN,NaN,NaN,1.0,21.0,6.0,0.750000
노유자시설,NaN,NaN,1.0,NaN,187.0,7.0,NaN,0.958974
단독주택,NaN,5.0,398.0,65.0,1.0,369.0,18474.0,0.956607
동.식물 관련시설,NaN,NaN,27.0,1157.0,NaN,6.0,128.0,0.877845


In [16]:
df2=df_nomi[['bldng_us_clssfctn','bldng_us']]
df2.dropna(inplace = True)
df_size = df2.pivot_table(index="bldng_us_clssfctn", columns="bldng_us", aggfunc="size")
df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False).iloc[0]/x.sum()),axis=1)
df_size

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


bldng_us,공동주택,공장,관광휴게시설,교육연구및복지시설,교육연구시설,근린생활시설,노유자시설,단독주택,동.식물 관련시설,묘지관련시설,...,위험물저장및처리시설,의료시설,자동차관련시설,장례식장,제1종근린생활시설,제2종근린생활시설,종교시설,창고시설,판매시설,percentage
bldng_us_clssfctn,,,,,,,,,,,,,,,,,,,,,
공공용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.588235
공업용,NaN,1032.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,1.0,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,0.990403
기타,3.0,29.0,NaN,NaN,1.0,NaN,1.0,398.0,27.0,NaN,...,NaN,NaN,NaN,NaN,9.0,4.0,NaN,1369.0,NaN,0.721286
농수산용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,1157.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,0.944490
문교사회용,NaN,NaN,20.0,16.0,174.0,1.0,187.0,1.0,NaN,9.0,...,NaN,27.0,NaN,3.0,6.0,10.0,145.0,1.0,NaN,0.276219
상업용,71.0,5.0,8.0,NaN,3.0,21.0,7.0,369.0,6.0,NaN,...,102.0,NaN,112.0,NaN,2152.0,2064.0,10.0,20.0,51.0,0.399406
주거용,858.0,1.0,NaN,NaN,NaN,6.0,NaN,18474.0,128.0,NaN,...,NaN,NaN,1.0,NaN,45.0,56.0,NaN,15.0,1.0,0.943177


In [17]:


#ttl_dwn__flr 0이 대부분임
df2=df_nomi[['emd_nm','ttl_dwn_flr']]
df2.dropna(inplace = True)
df_size = df2.pivot_table(index="emd_nm", columns="ttl_dwn_flr", aggfunc="size")
df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False)[0]/x.sum()),axis=1)
df_size

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


ttl_dwn_flr,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,23.0,30.0,31.0,33.0,36.0,42.0,46.0,48.0,60.0,percentage
emd_nm,,,,,,,,,,,,,,,,,,,,,
경상남도 거제시 거제면,251.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980469
경상남도 거제시 거제면 법동리,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
경상남도 거제시 거제면 오수리,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
경상남도 거제시 고현동,240.0,59.0,6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.781759
경상남도 거제시 남부면,93.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.958763
경상남도 거제시 능포동,79.0,9.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.858696
경상남도 거제시 동부면,205.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.957944
경상남도 거제시 둔덕면,151.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.993421
경상남도 거제시 둔덕면 학산리,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [18]:
#층이 높아지면 대부분 철근콘크리트구조임

#df2=df_nomi[['floors' ,'bldng_archtctr']]
#df2.dropna(inplace = True)
#df_size = df2.pivot_table(index="floors", columns="bldng_archtctr", aggfunc="size")
#df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False)[0]/x.sum()),axis=1)
#df_size
plot_heat(df_nomi,['floors'],['bldng_archtctr'])[2]

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


bldng_archtctr,강파이프구조,경량철골구조,기타강구조,기타구조,기타조적구조,기타콘크리트구조,목구조,벽돌구조,블록구조,석구조,...,일반철골구조,조적구조,철골철근콘크리트구조,철골콘크리트구조,철근콘크리트구조,통나무구조,프리케스트콘크리트구조,sum,percentage,score
floors,,,,,,,,,,,,,,,,,,,,,
0.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,7.0,13.0,NaN,...,1.0,2.0,NaN,NaN,5.0,NaN,NaN,44.0,0.295455,-23.298410
1.0,237.0,2005.0,21.0,14.0,110.0,NaN,14.0,3424.0,2805.0,37.0,...,611.0,215.0,2.0,10.0,738.0,4.0,NaN,15599.0,0.343099,-7246.993611
2.0,57.0,585.0,7.0,6.0,30.0,1.0,2.0,2633.0,681.0,12.0,...,527.0,152.0,2.0,7.0,1477.0,4.0,7.0,6948.0,0.378958,-2927.949389
3.0,16.0,168.0,3.0,2.0,5.0,NaN,NaN,728.0,135.0,2.0,...,239.0,15.0,3.0,5.0,1292.0,1.0,4.0,2812.0,0.459459,-949.760881
4.0,12.0,86.0,NaN,1.0,5.0,NaN,NaN,102.0,53.0,1.0,...,126.0,4.0,4.0,6.0,1174.0,1.0,2.0,1645.0,0.713678,-240.988890
5.0,8.0,60.0,1.0,NaN,3.0,NaN,1.0,17.0,21.0,NaN,...,67.0,2.0,5.0,1.0,469.0,NaN,1.0,676.0,0.693787,-107.331125
6.0,1.0,22.0,NaN,NaN,1.0,NaN,NaN,18.0,12.0,NaN,...,58.0,NaN,4.0,1.0,275.0,NaN,NaN,407.0,0.675676,-69.296518
7.0,2.0,19.0,NaN,NaN,4.0,NaN,NaN,8.0,7.0,NaN,...,39.0,NaN,3.0,NaN,141.0,NaN,NaN,228.0,0.618421,-47.587187
8.0,3.0,9.0,NaN,NaN,1.0,NaN,NaN,4.0,1.0,NaN,...,27.0,1.0,2.0,NaN,88.0,NaN,NaN,140.0,0.628571,-28.230351


In [19]:
#df2=df_nomi[['floors' ,'lnd_us_sttn_nm']]
#df2.dropna(inplace = True)
#df_size = df2.pivot_table(index="floors", columns="lnd_us_sttn_nm", aggfunc="size")
#df_size['percentage']=df_size.apply((lambda x: x.sort_values(ascending=False)[0]/x.sum()),axis=1)
#df_size
plot_heat(df_nomi,['floors'],['lnd_us_sttn_nm'])[2]

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


lnd_us_sttn_nm,고속도로휴게소,골프장 대중제,골프장 회원제,공업기타,공업나지,공업용,공원등,공원묘지,과수원,기타,...,주상기타,주상나지,주상용,주차장등,콘도미니엄,토지임야,하천등,sum,percentage,score
floors,,,,,,,,,,,,,,,,,,,,,
0.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,NaN,4.0,NaN,NaN,NaN,NaN,96.0,0.708333,-14.377183
1.0,NaN,NaN,1.0,44.0,24.0,403.0,8.0,NaN,83.0,7.0,...,142.0,28.0,599.0,9.0,NaN,28.0,9.0,26711.0,0.767175,-3074.591027
2.0,NaN,NaN,NaN,39.0,9.0,390.0,8.0,NaN,16.0,6.0,...,109.0,20.0,898.0,2.0,NaN,12.0,4.0,10589.0,0.644915,-2017.181581
3.0,1.0,NaN,NaN,18.0,12.0,268.0,1.0,NaN,2.0,3.0,...,64.0,96.0,658.0,4.0,NaN,2.0,1.0,4042.0,0.417368,-1533.863118
4.0,1.0,NaN,NaN,10.0,4.0,176.0,1.0,NaN,NaN,3.0,...,63.0,16.0,363.0,4.0,NaN,NaN,NaN,2350.0,0.292340,-1255.161144
5.0,NaN,NaN,NaN,4.0,15.0,117.0,1.0,NaN,3.0,3.0,...,13.0,8.0,131.0,1.0,NaN,NaN,NaN,975.0,0.290256,-523.787726
6.0,NaN,NaN,NaN,4.0,6.0,84.0,2.0,NaN,NaN,3.0,...,10.0,5.0,48.0,5.0,NaN,NaN,1.0,578.0,0.304498,-298.487969
7.0,NaN,NaN,NaN,NaN,8.0,63.0,NaN,1.0,NaN,NaN,...,9.0,NaN,17.0,NaN,NaN,NaN,NaN,329.0,0.261398,-191.707460
8.0,NaN,NaN,NaN,1.0,4.0,38.0,NaN,NaN,NaN,NaN,...,1.0,NaN,13.0,1.0,NaN,NaN,NaN,199.0,0.221106,-130.424680


# --------------------------------------------------------------------------------------------------------

In [44]:
a,b = find_2(df_nomi,'bldng_us_clssfctn')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-0.3553043233033945


bldng_us_clssfctn      공공용  공업용  기타  농수산용  문교사회용  상업용   주거용   sum  percentage  \
bldng_us emd_nm                                                                 
공동주택     경상남도 거제시 거제면  NaN  NaN NaN   NaN    NaN  NaN   5.0   5.0    1.000000   
         경상남도 거제시 고현동  NaN  NaN NaN   NaN    NaN  4.0  36.0  40.0    0.900000   
         경상남도 거제시 능포동  NaN  NaN NaN   NaN    NaN  3.0  11.0  14.0    0.785714   
         경상남도 거제시 동부면  NaN  NaN NaN   NaN    NaN  1.0   NaN   1.0    1.000000   
         경상남도 거제시 사등면  NaN  NaN NaN   NaN    NaN  2.0   1.0   3.0    0.666667   
...                    ...  ...  ..   ...    ...  ...   ...   ...         ...   
판매시설     경상남도 통영시 정량동  NaN  NaN NaN   NaN    NaN  1.0   NaN   1.0    1.000000   
         경상남도 하동군 하동읍  NaN  NaN NaN   NaN    NaN  1.0   NaN   1.0    1.000000   
         경상남도 함안군 가야읍  NaN  NaN NaN   NaN    NaN  4.0   NaN   4.0    1.000000   
         경상남도 합천군 삼가면  NaN  NaN NaN   NaN    NaN  2.0   NaN   2.0    1.000000   
         경상남도 합천군 합천읍  NaN  NaN NaN   NaN    NaN  1.0   NaN   1.0    1.000000   

bldng_us_clssfctn         score  
bldng_us emd_nm                  
공동주택     경상남도 거제시 거제면  0.000000  
         경상남도 거제시 고현동 -1.830300  
         경상남도 거제시 능포동 -1.466295  
         경상남도 거제시 동부면       NaN  
         경상남도 거제시 사등면 -0.528274  
...                         ...  
판매시설     경상남도 통영시 정량동       NaN  
         경상남도 하동군 하동읍       NaN  
         경상남도 함안군 가야읍  0.000000  
         경상남도 합천군 삼가면  0.000000  
         경상남도 합천군 합천읍       NaN  

[2561 rows x 10 columns]

In [69]:
print(df[(df['emd_nm'].notnull())&(df['bldng_us'].notnull())&(df['bldng_us_clssfctn'].isna())].shape)
print(df['bldng_us_clssfctn'].isna().sum())

(1671, 187)
29343


In [46]:
a,b = find_2(df_nomi,'bldng_us')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-1.9183780236888266


bldng_us                      공공용시설  공동주택  공장  관광휴게시설  교육연구및복지시설  교육연구시설  \
lnd_us_sttn_nm emd_nm                                                      
고속도로휴게소        경상남도 사천시 곤양면     NaN   NaN NaN     1.0        NaN     NaN   
               경상남도 진주시 가호동     NaN   NaN NaN     1.0        NaN     NaN   
               경상남도 진주시 문산읍     NaN   NaN NaN     3.0        NaN     NaN   
               경상남도 창녕군 영산면     NaN   NaN NaN     2.0        NaN     NaN   
               경상남도 함안군 군북면     NaN   NaN NaN     2.0        NaN     NaN   
...                             ...   ...  ..     ...        ...     ...   
하천등            경상남도 밀양시 교동      NaN   NaN NaN     NaN        NaN     NaN   
               경상남도 밀양시 삼랑진읍    NaN   NaN NaN     NaN        NaN     NaN   
               경상남도 양산시 양주동     NaN   NaN NaN     NaN        NaN     NaN   
               경상남도 창녕군 남지읍     NaN   NaN NaN     NaN        NaN     NaN   
               경상남도 함양군 안의면     NaN   NaN NaN     NaN        NaN     NaN   

bldng_us                      교정및군사시설  근린생활시설  노유자시설  단독주택  ...  제1종근린생활시설  \
lnd_us_sttn_nm emd_nm                                       ...              
고속도로휴게소        경상남도 사천시 곤양면       NaN     NaN    NaN   NaN  ...        NaN   
               경상남도 진주시 가호동       NaN     NaN    NaN   NaN  ...        NaN   
               경상남도 진주시 문산읍       NaN     NaN    NaN   NaN  ...        NaN   
               경상남도 창녕군 영산면       NaN     NaN    NaN   NaN  ...        NaN   
               경상남도 함안군 군북면       NaN     NaN    NaN   NaN  ...        NaN   
...                               ...     ...    ...   ...  ...        ...   
하천등            경상남도 밀양시 교동        NaN     NaN    NaN   NaN  ...        1.0   
               경상남도 밀양시 삼랑진읍      NaN     NaN    NaN   NaN  ...        1.0   
               경상남도 양산시 양주동       NaN     NaN    NaN   NaN  ...        1.0   
               경상남도 창녕군 남지읍       NaN     NaN    NaN   1.0  ...        NaN   
               경상남도 함양군 안의면       NaN     NaN    NaN   1.0  ...        NaN   

bldng_us                      제2종근린생활시설  종교시설  창고시설  파출소  판매및영업시설  판매시설  sum  \
lnd_us_sttn_nm emd_nm                                                          
고속도로휴게소        경상남도 사천시 곤양면         NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 진주시 가호동         NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 진주시 문산읍         NaN   NaN   NaN  NaN      NaN   NaN  3.0   
               경상남도 창녕군 영산면         NaN   NaN   NaN  NaN      NaN   NaN  2.0   
               경상남도 함안군 군북면         NaN   NaN   NaN  NaN      NaN   NaN  2.0   
...                                 ...   ...   ...  ...      ...   ...  ...   
하천등            경상남도 밀양시 교동          NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 밀양시 삼랑진읍        NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 양산시 양주동         NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 창녕군 남지읍         NaN   NaN   NaN  NaN      NaN   NaN  1.0   
               경상남도 함양군 안의면         NaN   NaN   NaN  NaN      NaN   NaN  1.0   

bldng_us                      percentage  score  
lnd_us_sttn_nm emd_nm                            
고속도로휴게소        경상남도 사천시 곤양면          1.0    NaN  
               경상남도 진주시 가호동          1.0    NaN  
               경상남도 진주시 문산읍          1.0    0.0  
               경상남도 창녕군 영산면          1.0    0.0  
               경상남도 함안군 군북면          1.0    0.0  
...                                  ...    ...  
하천등            경상남도 밀양시 교동           1.0    NaN  
               경상남도 밀양시 삼랑진읍         1.0    NaN  
               경상남도 양산시 양주동          1.0    NaN  
               경상남도 창녕군 남지읍          1.0    NaN  
               경상남도 함양군 안의면          1.0    NaN  

[3063 rows x 37 columns]

In [70]:
print(df[(df['emd_nm'].notnull())&(df['lnd_us_sttn_nm'].notnull())&(df['bldng_us'].isna())].shape)
print(df['bldng_us'].isna().sum())

(26578, 187)
27677


In [48]:
a,b = find_2(df_nomi,'bldng_archtctr')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-4.607019582451146


bldng_archtctr               강파이프구조  경량철골구조  기타강구조  기타구조  기타조적구조  기타콘크리트구조  \
emd_nm       lnd_us_sttn_nm                                                  
경상남도 거제시 거제면 공업용                NaN     NaN    NaN   NaN     NaN       NaN   
             다세대                NaN     NaN    NaN   NaN     NaN       NaN   
             단독                 NaN     7.0    NaN   NaN     1.0       NaN   
             답                  NaN     NaN    NaN   NaN     1.0       NaN   
             답기타                NaN     NaN    NaN   NaN     NaN       NaN   
...                             ...     ...    ...   ...     ...       ...   
경상남도 합천군 합천읍 전기타                NaN     1.0    NaN   NaN     NaN       NaN   
             주거기타               NaN     1.0    NaN   NaN     NaN       NaN   
             주거나지               NaN     NaN    NaN   NaN     NaN       NaN   
             주상기타               NaN     NaN    NaN   NaN     NaN       NaN   
             주상용                NaN     NaN    NaN   NaN     NaN       NaN   

bldng_archtctr               목구조  벽돌구조  블록구조  석구조  ...  일반철골구조  조적구조  \
emd_nm       lnd_us_sttn_nm                        ...                 
경상남도 거제시 거제면 공업용             NaN   NaN   NaN  NaN  ...     1.0   NaN   
             다세대             NaN   NaN   NaN  NaN  ...     NaN   NaN   
             단독              NaN  26.0  34.0  4.0  ...     2.0   NaN   
             답               NaN   NaN   NaN  NaN  ...     NaN   NaN   
             답기타             NaN   NaN   1.0  NaN  ...     NaN   NaN   
...                          ...   ...   ...  ...  ...     ...   ...   
경상남도 합천군 합천읍 전기타             NaN   NaN   NaN  NaN  ...     NaN   NaN   
             주거기타            NaN   3.0   1.0  NaN  ...     1.0   NaN   
             주거나지            NaN   NaN   NaN  NaN  ...     NaN   NaN   
             주상기타            NaN   NaN   NaN  NaN  ...     NaN   NaN   
             주상용             NaN   4.0   2.0  NaN  ...     1.0   NaN   

bldng_archtctr               철골철근콘크리트구조  철골콘크리트구조  철근콘크리트구조  통나무구조  \
emd_nm       lnd_us_sttn_nm                                          
경상남도 거제시 거제면 공업용                    NaN       NaN       NaN    NaN   
             다세대                    NaN       NaN       3.0    NaN   
             단독                     NaN       NaN      18.0    NaN   
             답                      NaN       NaN       NaN    NaN   
             답기타                    NaN       NaN       NaN    NaN   
...                                 ...       ...       ...    ...   
경상남도 합천군 합천읍 전기타                    NaN       NaN       NaN    NaN   
             주거기타                   NaN       NaN       7.0    NaN   
             주거나지                   NaN       NaN       2.0    NaN   
             주상기타                   NaN       NaN       2.0    NaN   
             주상용                    NaN       NaN       6.0    NaN   

bldng_archtctr               프리케스트콘크리트구조    sum  percentage      score  
emd_nm       lnd_us_sttn_nm                                             
경상남도 거제시 거제면 공업용                     NaN    1.0    1.000000        NaN  
             다세대                     NaN    3.0    1.000000   0.000000  
             단독                      NaN  120.0    0.283333 -65.724279  
             답                       NaN    1.0    1.000000        NaN  
             답기타                     NaN    1.0    1.000000        NaN  
...                                  ...    ...         ...        ...  
경상남도 합천군 합천읍 전기타                     NaN    1.0    1.000000        NaN  
             주거기타                    NaN   14.0    0.500000  -4.214420  
             주거나지                    NaN    4.0    0.500000  -1.204120  
             주상기타                    NaN    2.0    1.000000   0.000000  
             주상용                     NaN   14.0    0.428571  -5.151675  

[3065 rows x 21 columns]

In [71]:
print(df[(df['emd_nm'].notnull())&(df['lnd_us_sttn_nm'].notnull())&(df['bldng_archtctr'].isna())].shape)
print(df['bldng_archtctr'].isna().sum())

(26567, 187)
27665


In [50]:
df_temp = df_nomi.drop(['ttl_dwn_flr','ttl_grnd_flr'],axis=1)
a,b = find_2(df_temp,'floors')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-3.908985580312728


floors                          0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  \
bldng_archtctr emd_nm                                                         
강파이프구조         경상남도 거제시 남부면     NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 거제시 동부면     NaN  1.0  1.0  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 거제시 둔덕면     NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 거제시 사등면     NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 거제시 연초면     NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
프리케스트콘크리트구조    경상남도 진주시 일반성면    NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 진주시 정촌면     NaN  NaN  2.0  NaN  NaN  NaN  NaN  NaN  NaN   
               경상남도 창원시의창구 용지동  NaN  NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN   
               경상남도 창원시진해구 석동   NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN   
               경상남도 창원시진해구 이동   NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN  NaN   

floors                          9.0  ...  321.0  327.0  366.0  387.0  398.0  \
bldng_archtctr emd_nm                ...                                      
강파이프구조         경상남도 거제시 남부면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 거제시 동부면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 거제시 둔덕면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 거제시 사등면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 거제시 연초면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
...                             ...  ...    ...    ...    ...    ...    ...   
프리케스트콘크리트구조    경상남도 진주시 일반성면    NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 진주시 정촌면     NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 창원시의창구 용지동  NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 창원시진해구 석동   NaN  ...    NaN    NaN    NaN    NaN    NaN   
               경상남도 창원시진해구 이동   NaN  ...    NaN    NaN    NaN    NaN    NaN   

floors                          415.0  636.0  sum  percentage    score  
bldng_archtctr emd_nm                                                   
강파이프구조         경상남도 거제시 남부면       NaN    NaN  1.0         1.0      NaN  
               경상남도 거제시 동부면       NaN    NaN  2.0         0.5 -0.60206  
               경상남도 거제시 둔덕면       NaN    NaN  1.0         1.0      NaN  
               경상남도 거제시 사등면       NaN    NaN  1.0         1.0      NaN  
               경상남도 거제시 연초면       NaN    NaN  1.0         1.0      NaN  
...                               ...    ...  ...         ...      ...  
프리케스트콘크리트구조    경상남도 진주시 일반성면      NaN    NaN  1.0         1.0      NaN  
               경상남도 진주시 정촌면       NaN    NaN  2.0         1.0  0.00000  
               경상남도 창원시의창구 용지동    NaN    NaN  1.0         1.0      NaN  
               경상남도 창원시진해구 석동     NaN    NaN  1.0         1.0      NaN  
               경상남도 창원시진해구 이동     NaN    NaN  1.0         1.0      NaN  

[2067 rows x 157 columns]

In [72]:
print(df[(df['bldng_archtctr'].notnull())&(df['emd_nm'].notnull())&(df['floors'].isna())].shape)
print(df['floors'].isna().sum())

(2121, 187)
11006


In [52]:
df_temp = df_nomi.drop(['floors'],axis=1)
a,b = find_2(df_temp,'ttl_dwn_flr')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-0.6078748771819585


ttl_dwn_flr                   0.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  \
bldng_cnt emd_nm                                                             
1         경상남도 거제시 거제면       74.0   1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
          경상남도 거제시 거제면 오수리    1.0   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
          경상남도 거제시 고현동      132.0  34.0  3.0  NaN  2.0  NaN  NaN  NaN  NaN   
          경상남도 거제시 남부면       27.0   4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
          경상남도 거제시 능포동       31.0   4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...                           ...   ...  ...  ...  ...  ...  ...  ...  ...   
138       경상남도 창원시성산구 웅남동     NaN   NaN  NaN  1.0  NaN  NaN  NaN  NaN  NaN   
148       경상남도 합천군 용주면        1.0   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
174       경상남도 창원시성산구 웅남동     NaN   NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN   
321       경상남도 진주시 가호동        NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
705       경상남도 창원시성산구 웅남동     NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

ttl_dwn_flr                 9.0  ...  31.0  33.0  36.0  42.0  46.0  48.0  \
bldng_cnt emd_nm                 ...                                       
1         경상남도 거제시 거제면      NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
          경상남도 거제시 거제면 오수리  NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
          경상남도 거제시 고현동      NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
          경상남도 거제시 남부면      NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
          경상남도 거제시 능포동      NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
...                         ...  ...   ...   ...   ...   ...   ...   ...   
138       경상남도 창원시성산구 웅남동   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
148       경상남도 합천군 용주면      NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
174       경상남도 창원시성산구 웅남동   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   
321       경상남도 진주시 가호동      NaN  ...   NaN   NaN   6.0   NaN   NaN   NaN   
705       경상남도 창원시성산구 웅남동   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   

ttl_dwn_flr                 60.0    sum  percentage      score  
bldng_cnt emd_nm                                                
1         경상남도 거제시 거제면       NaN   75.0    0.986667  -0.437216  
          경상남도 거제시 거제면 오수리   NaN    1.0    1.000000        NaN  
          경상남도 거제시 고현동       NaN  171.0    0.771930 -19.224193  
          경상남도 거제시 남부면       NaN   31.0    0.870968  -1.859936  
          경상남도 거제시 능포동       NaN   35.0    0.885714  -1.844722  
...                          ...    ...         ...        ...  
138       경상남도 창원시성산구 웅남동    NaN    1.0    1.000000        NaN  
148       경상남도 합천군 용주면       NaN    1.0    1.000000        NaN  
174       경상남도 창원시성산구 웅남동    NaN    1.0    1.000000        NaN  
321       경상남도 진주시 가호동       NaN    6.0    1.000000   0.000000  
705       경상남도 창원시성산구 웅남동    NaN    1.0    1.000000        NaN  

[3695 rows x 32 columns]

In [73]:
print(df[(df['bldng_cnt'].notnull())&(df['emd_nm'].notnull())&(df['ttl_dwn_flr'].isna())].shape)
print(df['ttl_dwn_flr'].isna().sum())

(11004, 187)
11005


In [54]:
df_temp = df_nomi.drop(['floors'],axis=1)
a,b = find_2(df_temp,'ttl_grnd_flr')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-3.6772613002565473


ttl_grnd_flr                 0.0   1.0   2.0   3.0  4.0  5.0  6.0  7.0  8.0  \
emd_nm       bldng_archtctr                                                   
경상남도 거제시 거제면 경량철골구조          NaN  10.0   3.0   1.0  NaN  NaN  NaN  NaN  NaN   
             기타강구조           NaN   NaN   1.0   NaN  NaN  NaN  NaN  NaN  NaN   
             기타조적구조          NaN   2.0   NaN   NaN  NaN  NaN  NaN  NaN  NaN   
             벽돌구조            NaN  23.0   4.0   1.0  NaN  NaN  NaN  NaN  NaN   
             블록구조            NaN  31.0   7.0   1.0  NaN  NaN  NaN  NaN  NaN   
...                          ...   ...   ...   ...  ...  ...  ...  ...  ...   
경상남도 합천군 합천읍 블록구조            NaN  16.0   3.0   NaN  NaN  NaN  NaN  NaN  NaN   
             일반목구조           NaN  52.0   5.0   3.0  NaN  1.0  NaN  NaN  NaN   
             일반철골구조          NaN   2.0   4.0   1.0  NaN  NaN  NaN  NaN  NaN   
             철골철근콘크리트구조      NaN   1.0   NaN   NaN  NaN  NaN  NaN  NaN  NaN   
             철근콘크리트구조        NaN   6.0  12.0  16.0  9.0  3.0  1.0  3.0  NaN   

ttl_grnd_flr                 9.0  ...  312.0  343.0  351.0  369.0  395.0  \
emd_nm       bldng_archtctr       ...                                      
경상남도 거제시 거제면 경량철골구조          NaN  ...    NaN    NaN    NaN    NaN    NaN   
             기타강구조           NaN  ...    NaN    NaN    NaN    NaN    NaN   
             기타조적구조          NaN  ...    NaN    NaN    NaN    NaN    NaN   
             벽돌구조            NaN  ...    NaN    NaN    NaN    NaN    NaN   
             블록구조            NaN  ...    NaN    NaN    NaN    NaN    NaN   
...                          ...  ...    ...    ...    ...    ...    ...   
경상남도 합천군 합천읍 블록구조            NaN  ...    NaN    NaN    NaN    NaN    NaN   
             일반목구조           NaN  ...    NaN    NaN    NaN    NaN    NaN   
             일반철골구조          NaN  ...    NaN    NaN    NaN    NaN    NaN   
             철골철근콘크리트구조      NaN  ...    NaN    NaN    NaN    NaN    NaN   
             철근콘크리트구조        NaN  ...    NaN    NaN    NaN    NaN    NaN   

ttl_grnd_flr                 413.0  594.0   sum  percentage      score  
emd_nm       bldng_archtctr                                             
경상남도 거제시 거제면 경량철골구조            NaN    NaN  14.0    0.714286  -2.045792  
             기타강구조             NaN    NaN   1.0    1.000000        NaN  
             기타조적구조            NaN    NaN   2.0    1.000000   0.000000  
             벽돌구조              NaN    NaN  28.0    0.821429  -2.392045  
             블록구조              NaN    NaN  39.0    0.794872  -3.888414  
...                            ...    ...   ...         ...        ...  
경상남도 합천군 합천읍 블록구조              NaN    NaN  20.0    0.800000  -1.938200  
             일반목구조             NaN    NaN  61.0    0.852459  -4.228916  
             일반철골구조            NaN    NaN   7.0    0.571429  -1.701266  
             철골철근콘크리트구조        NaN    NaN   1.0    1.000000        NaN  
             철근콘크리트구조          NaN    NaN  56.0    0.285714 -30.467810  

[2076 rows x 155 columns]

In [74]:
print(df[(df['bldng_archtctr'].notnull())&(df['emd_nm'].notnull())&(df['ttl_grnd_flr'].isna())].shape)
print(df['ttl_grnd_flr'].isna().sum())

(1635, 187)
10210


In [56]:
a,b = find_2(df_nomi,'lnd_us_sttn_nm')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-2.5435892299573912


lnd_us_sttn_nm         고속도로휴게소  골프장 대중제  골프장 회원제  공업기타  공업나지  공업용  공원등  공원묘지  \
bldng_us emd_nm                                                                
공공용시설    경상남도 창녕군 성산면      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
공동주택     경상남도 거제시 거제면      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 거제시 고현동      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 거제시 능포동      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 거제시 동부면      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
...                        ...      ...      ...   ...   ...  ...  ...   ...   
판매시설     경상남도 통영시 정량동      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 하동군 하동읍      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 함안군 가야읍      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 합천군 삼가면      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   
         경상남도 합천군 합천읍      NaN      NaN      NaN   NaN   NaN  NaN  NaN   NaN   

lnd_us_sttn_nm         과수원  기타  ...  주거나지  주상기타  주상나지  주상용  주차장등  토지임야  하천등  \
bldng_us emd_nm                 ...                                           
공공용시설    경상남도 창녕군 성산면  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
공동주택     경상남도 거제시 거제면  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
         경상남도 거제시 고현동  NaN NaN  ...   NaN   NaN   2.0  5.0   NaN   NaN  NaN   
         경상남도 거제시 능포동  NaN NaN  ...   NaN   NaN   NaN  1.0   NaN   NaN  NaN   
         경상남도 거제시 동부면  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
...                    ...  ..  ...   ...   ...   ...  ...   ...   ...  ...   
판매시설     경상남도 통영시 정량동  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
         경상남도 하동군 하동읍  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
         경상남도 함안군 가야읍  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
         경상남도 합천군 삼가면  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   
         경상남도 합천군 합천읍  NaN NaN  ...   NaN   NaN   NaN  NaN   NaN   NaN  NaN   

lnd_us_sttn_nm          sum  percentage     score  
bldng_us emd_nm                                    
공공용시설    경상남도 창녕군 성산면   1.0    1.000000       NaN  
공동주택     경상남도 거제시 거제면   5.0    0.600000 -1.109244  
         경상남도 거제시 고현동  40.0    0.725000 -5.586480  
         경상남도 거제시 능포동  14.0    0.857143 -0.937255  
         경상남도 거제시 동부면   1.0    1.000000       NaN  
...                     ...         ...       ...  
판매시설     경상남도 통영시 정량동   1.0    1.000000       NaN  
         경상남도 하동군 하동읍   1.0    1.000000       NaN  
         경상남도 함안군 가야읍   4.0    1.000000  0.000000  
         경상남도 합천군 삼가면   2.0    1.000000  0.000000  
         경상남도 합천군 합천읍   1.0    1.000000       NaN  

[2587 rows x 42 columns]

In [75]:
print(df[(df['bldng_us'].notnull())&(df['emd_nm'].notnull())&(df['lnd_us_sttn_nm'].isna())].shape)
print(df['lnd_us_sttn_nm'].isna().sum())

(675, 187)
1776


In [58]:
a,b = find_2(df_nomi,'rd_sd_nm')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-5.831242448925332


rd_sd_nm                광대로한면  광대세각  광대소각   맹지  세로각지(가)  세로각지(불)  세로한면(가)  \
emd_nm       bldng_us                                                       
경상남도 거제시 거제면 공동주택         NaN   NaN   NaN  NaN      NaN      NaN      3.0   
             공장           NaN   NaN   NaN  NaN      NaN      NaN      1.0   
             교육연구시설       NaN   NaN   NaN  NaN      NaN      NaN      2.0   
             노유자시설        NaN   NaN   NaN  NaN      NaN      NaN      1.0   
             단독주택         NaN   NaN   NaN  1.0      7.0      1.0     51.0   
...                       ...   ...   ...  ...      ...      ...      ...   
경상남도 합천군 합천읍 자동차관련시설      NaN   NaN   NaN  NaN      NaN      NaN      1.0   
             제1종근린생활시설    NaN   NaN   NaN  NaN      NaN      NaN      NaN   
             제2종근린생활시설    NaN   NaN   NaN  1.0      1.0      NaN      1.0   
             창고시설         NaN   NaN   NaN  NaN      NaN      NaN      2.0   
             판매시설         NaN   NaN   NaN  NaN      NaN      NaN      NaN   

rd_sd_nm                세로한면(불)  소로각지  소로한면  중로각지  중로한면  지정되지않음    sum  \
emd_nm       bldng_us                                                    
경상남도 거제시 거제면 공동주택           NaN   NaN   2.0   NaN   NaN     NaN    5.0   
             공장             NaN   NaN   1.0   NaN   NaN     NaN    2.0   
             교육연구시설         NaN   NaN   NaN   NaN   NaN     NaN    2.0   
             노유자시설          NaN   NaN   1.0   NaN   NaN     NaN    2.0   
             단독주택          51.0   1.0   9.0   NaN   NaN     NaN  121.0   
...                         ...   ...   ...   ...   ...     ...    ...   
경상남도 합천군 합천읍 자동차관련시설        NaN   NaN   NaN   NaN   NaN     NaN    1.0   
             제1종근린생활시설      1.0   NaN   9.0   3.0   4.0     NaN   17.0   
             제2종근린생활시설      4.0   1.0   8.0   2.0   3.0     NaN   21.0   
             창고시설           2.0   NaN   1.0   NaN   NaN     NaN    5.0   
             판매시설           NaN   1.0   NaN   NaN   NaN     NaN    1.0   

rd_sd_nm                percentage      score  
emd_nm       bldng_us                          
경상남도 거제시 거제면 공동주택         0.600000  -1.109244  
             공장           0.500000  -0.602060  
             교육연구시설       1.000000   0.000000  
             노유자시설        0.500000  -0.602060  
             단독주택         0.421488 -45.401039  
...                            ...        ...  
경상남도 합천군 합천읍 자동차관련시설      1.000000        NaN  
             제1종근린생활시설    0.529412  -4.695509  
             제2종근린생활시설    0.380952  -8.801715  
             창고시설         0.400000  -1.989700  
             판매시설         1.000000        NaN  

[2590 rows x 16 columns]

In [76]:
print(df[(df['bldng_us'].notnull())&(df['emd_nm'].notnull())&(df['rd_sd_nm'].isna())].shape)
print(df['rd_sd_nm'].isna().sum())

(527, 187)
1491


In [60]:
a,b = find_2(df_nomi,'rgnl_ar_nm2')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-0.10126809009569343


rgnl_ar_nm2            개발제한구역  계획관리지역  관리지역  농림지역  보전관리지역  보전녹지지역  생산관리지역  \
bldng_us emd_nm                                                             
공공용시설    경상남도 창녕군 성산면     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
공동주택     경상남도 거제시 거제면     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 거제시 고현동     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 거제시 능포동     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 거제시 동부면     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
...                       ...     ...   ...   ...     ...     ...     ...   
판매시설     경상남도 통영시 정량동     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 하동군 하동읍     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 함안군 가야읍     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 합천군 삼가면     NaN     NaN   NaN   NaN     NaN     NaN     NaN   
         경상남도 합천군 합천읍     NaN     NaN   NaN   NaN     NaN     NaN     NaN   

rgnl_ar_nm2            생산녹지지역  일반공업지역  일반상업지역  ...  자연환경보전지역  제1종일반주거지역  \
bldng_us emd_nm                                ...                        
공공용시설    경상남도 창녕군 성산면     NaN     NaN     NaN  ...       NaN        NaN   
공동주택     경상남도 거제시 거제면     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 거제시 고현동     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 거제시 능포동     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 거제시 동부면     NaN     NaN     NaN  ...       NaN        NaN   
...                       ...     ...     ...  ...       ...        ...   
판매시설     경상남도 통영시 정량동     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 하동군 하동읍     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 함안군 가야읍     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 합천군 삼가면     NaN     NaN     NaN  ...       NaN        NaN   
         경상남도 합천군 합천읍     NaN     NaN     NaN  ...       NaN        NaN   

rgnl_ar_nm2            제2종일반주거지역  제3종일반주거지역  준공업지역  준주거지역  지정되지않음   sum  \
bldng_us emd_nm                                                           
공공용시설    경상남도 창녕군 성산면        NaN        NaN    NaN    NaN     1.0   1.0   
공동주택     경상남도 거제시 거제면        NaN        NaN    NaN    NaN     5.0   5.0   
         경상남도 거제시 고현동        NaN        NaN    NaN    NaN    40.0  40.0   
         경상남도 거제시 능포동        NaN        NaN    NaN    NaN    14.0  14.0   
         경상남도 거제시 동부면        NaN        NaN    NaN    NaN     1.0   1.0   
...                          ...        ...    ...    ...     ...   ...   
판매시설     경상남도 통영시 정량동        NaN        NaN    NaN    NaN     1.0   1.0   
         경상남도 하동군 하동읍        NaN        NaN    NaN    NaN     1.0   1.0   
         경상남도 함안군 가야읍        NaN        NaN    NaN    NaN     4.0   4.0   
         경상남도 합천군 삼가면        NaN        NaN    NaN    NaN     2.0   2.0   
         경상남도 합천군 합천읍        NaN        NaN    NaN    NaN     1.0   1.0   

rgnl_ar_nm2            percentage  score  
bldng_us emd_nm                           
공공용시설    경상남도 창녕군 성산면         1.0    NaN  
공동주택     경상남도 거제시 거제면         1.0    0.0  
         경상남도 거제시 고현동         1.0    0.0  
         경상남도 거제시 능포동         1.0    0.0  
         경상남도 거제시 동부면         1.0    NaN  
...                           ...    ...  
판매시설     경상남도 통영시 정량동         1.0    NaN  
         경상남도 하동군 하동읍         1.0    NaN  
         경상남도 함안군 가야읍         1.0    0.0  
         경상남도 합천군 삼가면         1.0    0.0  
         경상남도 합천군 합천읍         1.0    NaN  

[2590 rows x 21 columns]

In [77]:
print(df[(df['bldng_us'].notnull())&(df['emd_nm'].notnull())&(df['rgnl_ar_nm2'].isna())].shape)
print(df['rgnl_ar_nm2'].isna().sum())

(527, 187)
1491


In [62]:
a,b = find_2(df_nomi,'rgnl_ar_nm')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-3.246128857375791


rgnl_ar_nm             개발제한구역  계획관리지역  관리지역  근린상업지역  농림지역  보전관리지역  보전녹지지역  \
bldng_us emd_nm                                                             
공공용시설    경상남도 창녕군 성산면     NaN     1.0   NaN     NaN   NaN     NaN     NaN   
공동주택     경상남도 거제시 거제면     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 거제시 고현동     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 거제시 능포동     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 거제시 동부면     NaN     1.0   NaN     NaN   NaN     NaN     NaN   
...                       ...     ...   ...     ...   ...     ...     ...   
판매시설     경상남도 통영시 정량동     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 하동군 하동읍     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 함안군 가야읍     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 합천군 삼가면     NaN     NaN   NaN     NaN   NaN     NaN     NaN   
         경상남도 합천군 합천읍     NaN     NaN   NaN     NaN   NaN     NaN     NaN   

rgnl_ar_nm             생산관리지역  생산녹지지역  용도미지정  ...  제1종전용주거지역  제2종일반주거지역  \
bldng_us emd_nm                               ...                         
공공용시설    경상남도 창녕군 성산면     NaN     NaN    NaN  ...        NaN        NaN   
공동주택     경상남도 거제시 거제면     NaN     NaN    NaN  ...        NaN        5.0   
         경상남도 거제시 고현동     NaN     NaN    NaN  ...        NaN       37.0   
         경상남도 거제시 능포동     NaN     NaN    NaN  ...        NaN       14.0   
         경상남도 거제시 동부면     NaN     NaN    NaN  ...        NaN        NaN   
...                       ...     ...    ...  ...        ...        ...   
판매시설     경상남도 통영시 정량동     NaN     NaN    NaN  ...        NaN        1.0   
         경상남도 하동군 하동읍     NaN     1.0    NaN  ...        NaN        NaN   
         경상남도 함안군 가야읍     NaN     NaN    NaN  ...        NaN        NaN   
         경상남도 합천군 삼가면     NaN     NaN    NaN  ...        NaN        NaN   
         경상남도 합천군 합천읍     NaN     NaN    NaN  ...        NaN        NaN   

rgnl_ar_nm             제2종전용주거지역  제3종일반주거지역  준공업지역  준주거지역  중심상업지역   sum  \
bldng_us emd_nm                                                           
공공용시설    경상남도 창녕군 성산면        NaN        NaN    NaN    NaN     NaN   1.0   
공동주택     경상남도 거제시 거제면        NaN        NaN    NaN    NaN     NaN   5.0   
         경상남도 거제시 고현동        NaN        NaN    NaN    NaN     NaN  40.0   
         경상남도 거제시 능포동        NaN        NaN    NaN    NaN     NaN  14.0   
         경상남도 거제시 동부면        NaN        NaN    NaN    NaN     NaN   1.0   
...                          ...        ...    ...    ...     ...   ...   
판매시설     경상남도 통영시 정량동        NaN        NaN    NaN    NaN     NaN   1.0   
         경상남도 하동군 하동읍        NaN        NaN    NaN    NaN     NaN   1.0   
         경상남도 함안군 가야읍        NaN        NaN    NaN    NaN     NaN   4.0   
         경상남도 합천군 삼가면        NaN        NaN    NaN    NaN     NaN   2.0   
         경상남도 합천군 합천읍        NaN        NaN    NaN    NaN     NaN   1.0   

rgnl_ar_nm             percentage     score  
bldng_us emd_nm                              
공공용시설    경상남도 창녕군 성산면       1.000       NaN  
공동주택     경상남도 거제시 거제면       1.000  0.000000  
         경상남도 거제시 고현동       0.925 -1.354331  
         경상남도 거제시 능포동       1.000  0.000000  
         경상남도 거제시 동부면       1.000       NaN  
...                           ...       ...  
판매시설     경상남도 통영시 정량동       1.000       NaN  
         경상남도 하동군 하동읍       1.000       NaN  
         경상남도 함안군 가야읍       1.000  0.000000  
         경상남도 합천군 삼가면       1.000  0.000000  
         경상남도 합천군 합천읍       1.000       NaN  

[2590 rows x 26 columns]

In [78]:
print(df[(df['bldng_us'].notnull())&(df['emd_nm'].notnull())&(df['rgnl_ar_nm'].isna())].shape)
print(df['rgnl_ar_nm'].isna().sum())

(527, 187)
1491


In [64]:
a,b = find_2(df_nomi,'emd_nm')
print(a)
b

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-136.8436385159471


emd_nm           경상남도 거제시 거제면  경상남도 거제시 거제면 오수리  경상남도 거제시 고현동  경상남도 거제시 남부면  \
bldng_us floors                                                               
공동주택     1.0              NaN               NaN           NaN           NaN   
         2.0              NaN               NaN           NaN           NaN   
         3.0              NaN               NaN           3.0           NaN   
         4.0              1.0               NaN           1.0           NaN   
         5.0              2.0               NaN           3.0           NaN   
...                       ...               ...           ...           ...   
판매시설     6.0              NaN               NaN           NaN           NaN   
         7.0              NaN               NaN           NaN           NaN   
         8.0              NaN               NaN           NaN           NaN   
         13.0             NaN               NaN           NaN           NaN   
         14.0             NaN               NaN           NaN           NaN   

emd_nm           경상남도 거제시 능포동  경상남도 거제시 동부면  경상남도 거제시 둔덕면  경상남도 거제시 둔덕면 학산리  \
bldng_us floors                                                               
공동주택     1.0              NaN           NaN           NaN               NaN   
         2.0              NaN           NaN           NaN               NaN   
         3.0              2.0           NaN           NaN               NaN   
         4.0              1.0           NaN           NaN               NaN   
         5.0              NaN           NaN           NaN               NaN   
...                       ...           ...           ...               ...   
판매시설     6.0              NaN           NaN           NaN               NaN   
         7.0              NaN           NaN           NaN               NaN   
         8.0              NaN           NaN           NaN               NaN   
         13.0             NaN           NaN           NaN               NaN   
         14.0             NaN           NaN           NaN               NaN   

emd_nm           경상남도 거제시 사등면  경상남도 거제시 상문동  ...  경상남도 합천군 야로면  경상남도 합천군 용주면  \
bldng_us floors                              ...                               
공동주택     1.0              NaN           NaN  ...           1.0           NaN   
         2.0              NaN           NaN  ...           NaN           NaN   
         3.0              NaN           NaN  ...           NaN           NaN   
         4.0              1.0           NaN  ...           NaN           NaN   
         5.0              NaN           NaN  ...           NaN           NaN   
...                       ...           ...  ...           ...           ...   
판매시설     6.0              NaN           NaN  ...           NaN           NaN   
         7.0              NaN           NaN  ...           NaN           NaN   
         8.0              NaN           NaN  ...           NaN           NaN   
         13.0             NaN           NaN  ...           NaN           NaN   
         14.0             NaN           NaN  ...           NaN           NaN   

emd_nm           경상남도 합천군 율곡면  경상남도 합천군 적중면  경상남도 합천군 청덕면  경상남도 합천군 초계면  \
bldng_us floors                                                           
공동주택     1.0              NaN           NaN           NaN           NaN   
         2.0              NaN           NaN           NaN           NaN   
         3.0              NaN           NaN           NaN           NaN   
         4.0              NaN           NaN           NaN           NaN   
         5.0              NaN           NaN           NaN           NaN   
...                       ...           ...           ...           ...   
판매시설     6.0              NaN           NaN           NaN           NaN   
         7.0              NaN           NaN           NaN           NaN   
         8.0              NaN           NaN           NaN           NaN   
         13.0             NaN           NaN           NaN           NaN   
   

In [79]:
print(df[(df['bldng_us'].notnull())&(df['floors'].notnull())&(df['emd_nm'].isna())].shape)
print(df['emd_nm'].isna().sum())

(2, 187)
4


In [14]:
x=np.nan
if np.isnan(x):
    print('hi')

hi
